In [15]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query
import preprocesser
import datetime
import pandas.tseries.offsets as of  # for timeoffset

In [16]:
def non_decreasing(L):
    data = [x<=y for x, y in zip(L, L[1:])]
    return data

In [17]:
#1.종목명
shcode_list = query.query_shcode_list_today()

#2.target from csvs
df = pd.read_csv("high_ratio_0.05_volume_money_20000000000_v3.csv", dtype ={'shcode':str})

#3. target index 추출
fin_result = pd.DataFrame([],columns = ['target_day','shcode'])

for i,v in df['index'].items():
    shcode = df['shcode'].iloc[i].zfill(6)
    target_date = v.split(" ")
    target_day_list = pd.to_datetime(target_date, format = "%Y-%m-%d")
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=6)
    result = pd.DataFrame(dict(target_day = target_day_list.strftime("%Y%m%d"),
                              interval_end = interval_end_timestamp.strftime("%Y%m%d"))).reset_index()
    del result['index']
    result['shcode'] = shcode
    fin_result = fin_result.append(result)
    
#4. make target month
fin_result['target_m'] = fin_result['target_day'].str.slice(stop = 6)
fin_result = pd.merge(fin_result, shcode_list, how = 'inner', on = 'shcode')

#5. target day 확인
target_day_list = pd.to_datetime(fin_result['target_day'], format = "%Y-%m-%d")

#(1) 월
# bf
#(2) 주
# bf
#(3) 일
check_init_timestamp = target_day_list - datetime.timedelta(days=60)
fin_result['check_init_day'] = check_init_timestamp.apply(lambda x: x.strftime("%Y%m%d"))

#for 크로스전 for 3번째전략
check_init_timestamp2 = target_day_list - datetime.timedelta(days=30)
fin_result['check_init_day2'] = check_init_timestamp2.apply(lambda x: x.strftime("%Y%m%d"))

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [18]:
fin_result = fin_result[fin_result['target_m']>='201909'].reset_index()
del fin_result['index']
fin_result['index'] = fin_result.index

In [14]:
    #for result
    '''
    # 5, 20, gap 상승안하면 안한다.
    if ma5_score * ma20_score * ma5_20_score == 0:
        continue
    # 1년내 10000원이하는 안한다.
    #elif df_weekly['high'].iloc[-12-3:-3].max() <= 10000:
    #    continue
    # 직전 12개월동안 최대 max 통과 안할시 안한다 악성매물
    elif df_weekly['high'].iloc[-3] <= df_weekly['high'].iloc[-12-3:-3].max():
        continue
    '''

"\n# 5, 20, gap 상승안하면 안한다.\nif ma5_score * ma20_score * ma5_20_score == 0:\n    continue\n# 1년내 10000원이하는 안한다.\n#elif df_weekly['high'].iloc[-12-3:-3].max() <= 10000:\n#    continue\n# 직전 12개월동안 최대 max 통과 안할시 안한다 악성매물\nelif df_weekly['high'].iloc[-3] <= df_weekly['high'].iloc[-12-3:-3].max():\n    continue\n"

In [19]:
rrr = pd.DataFrame([],columns = ['date',  'close_ratio', 'min_target', 'max_target', 'open_', 'low', 'high',
                                 'close','volume' ,'volume_ratio', 'range_position', 'spot_range', 'inrange', 'spot_range_inc',
                                 'squeezed', 'volume_default', 'deal_f','burn_ratio','rotate_ratio', 
                                 'for_net_buy','com_net_buy','per_net_buy',
                                 'shcode','shname', 'target_m', 'index'
                                ])

In [20]:
for i,v in fin_result['shcode'].items():
    #(3) 일별확인
    #init 재조정    
    df_init = query.query_t1305(shcode = fin_result['shcode'].iloc[i], 
                                fr = fin_result['check_init_day'].iloc[i], 
                                to = fin_result['target_day'].iloc[i])
    range_init = df_init['close'].iloc[-2]
    for j,w in df_init['close'].iloc[::-1].iloc[1:].items():
        if w <= range_init:
            range_init = w
            continue
        else:
            break
            
    df = query.query_t1305(shcode = fin_result['shcode'].iloc[i],
                           fr = fin_result['target_day'].iloc[i], 
                           to = fin_result['interval_end'].iloc[i])    
    df['volume_money'] = df['close'] * df['volume']
    target_volume = df['volume'].iloc[0]
    range_end = df['high'].iloc[0]
    
    #close 재조정
    for j,w in df['high'].iloc[1:].items():
        if w >= range_end:
            range_end = w
            continue
        else:
            break
    
    # 눌림목 발생하고     
    sqeezed_bool = abs(df['close'] /df['open'] - 1) <= 0.05
    #거래량 10/100 망하고,
    volume_bool = df['volume'] / target_volume <= 0.1
    # range 사이의 범위
    spot_range_value = (df['close'] - range_init) / (range_end - range_init)
    # inrange 여부
    inrange_bool = (df['close'] >= range_init) & (df['close'] <= range_end)    
    # spot_range 상승여부
    spot_range_inc_bool = (spot_range_value - spot_range_value.shift(1)) >= 0.03
    #종가상승률
    close_ratio_list = [y/x-1 for x, y in zip(np.array(df)[:,4][:-1], np.array(df)[:,4][1:])]
    close_ratio_list.insert(0,0)
    
    test_list = pd.DataFrame(dict(date = np.array(df)[:,0],
                                  close_ratio = close_ratio_list,
                                  min_target = range_init,
                                  max_target = range_end,
                                  open_ = df['open'],
                                  low = df['low'],
                                  high = df['high'],
                                  close = df['close'],
                                  volume = df['volume'],
                                  volume_ratio = df['volume'] / target_volume,
                                  range_position = (df['close']-df['low'])/(df['high']-df['low']),
                                  spot_range = spot_range_value,
                                  inrange = inrange_bool,
                                  spot_range_inc = spot_range_inc_bool,
                                  squeezed = sqeezed_bool,                                  
                                  volume_default = volume_bool,
                                  deal_f = df['deal_f'],
                                  burn_ratio = df['burn_ratio'],
                                  rotate_ratio = df['rotate_ratio'],
                                  for_net_buy = df['for_net_buy'],
                                  com_net_buy = df['com_net_buy'],
                                  per_net_buy = df['per_net_buy']
                                 ))
    
    test_list['shcode'] = fin_result['shcode'].iloc[i]
    test_list['shname'] = fin_result['shname'].iloc[i]
    test_list['target_m'] = fin_result['target_m'].iloc[i]
    test_list['index'] = i
    
    #if any(test_list['check']) == 1:
    rrr = rrr.append(test_list)
    print(i, v, '완료')

0 000230 완료
1 000400 완료
2 000885 완료
3 000890 완료
4 001140 완료
5 001210 완료
6 001210 완료
7 001340 완료
8 001360 완료
9 001360 완료
10 001360 완료
11 001440 완료
12 001790 완료
13 001790 완료
14 001820 완료
15 002020 완료
16 002070 완료
17 002070 완료
18 002070 완료
19 002100 완료
20 002140 완료
21 002140 완료
22 002210 완료
23 002360 완료
24 002720 완료
25 002720 완료
26 002760 완료
27 002760 완료
28 002995 완료
29 003060 완료
30 003060 완료
31 003060 완료
32 003280 완료
33 003280 완료
34 003350 완료
35 003350 완료
36 003560 완료
37 003580 완료
38 003580 완료
39 003580 완료
40 004100 완료
41 004710 완료
42 004770 완료
43 004870 완료
44 005070 완료
45 005070 완료
46 005110 완료
47 005110 완료
48 005360 완료
49 005390 완료
50 005390 완료
51 005690 완료
52 006090 완료
53 006120 완료
54 006125 완료
55 006280 완료
56 006490 완료
57 006490 완료
58 006840 완료
59 006980 완료
60 006980 완료
61 006980 완료
62 007110 완료
63 007210 완료
64 007460 완료
65 007460 완료
66 007460 완료
67 007460 완료
68 007660 완료
69 007810 완료
70 008350 완료
71 008350 완료
72 008350 완료
73 008350 완료
74 008355 완료
75 008600 완료
76 008600 완료
77 008730

594 225430 완료
595 225530 완료
596 226340 완료
597 226360 완료
598 232140 완료
599 234100 완료
600 236200 완료
601 238090 완료
602 238120 완료
603 238200 완료
604 238200 완료
605 238490 완료
606 239340 완료
607 241840 완료
608 242040 완료
609 244460 완료
610 246690 완료
611 246710 완료
612 246720 완료
613 246960 완료
614 250000 완료
615 250000 완료
616 251970 완료
617 253450 완료
618 253840 완료
619 260660 완료
620 260660 완료
621 263540 완료
622 263540 완료
623 263700 완료
624 263700 완료
625 263800 완료
626 268600 완료
627 268600 완료
628 277070 완료
629 277410 완료
630 288620 완료
631 288620 완료
632 290380 완료
633 290550 완료
634 290740 완료
635 293780 완료
636 298380 완료
637 298380 완료
638 299660 완료
639 299660 완료
640 300080 완료
641 303030 완료
642 303030 완료
643 307160 완료
644 307180 완료
645 307930 완료
646 311390 완료
647 311390 완료
648 313760 완료
649 317850 완료
650 317850 완료
651 317850 완료
652 317850 완료
653 318000 완료
654 318000 완료
655 900100 완료
656 900100 완료
657 900100 완료
658 900270 완료
659 900280 완료
660 900290 완료
661 900310 완료
662 900310 완료
663 900340 완료
664 900340 완료
665 90

In [21]:
df_for = fin_result[['index','shcode','shname','target_day']]
result_for_draw = pd.DataFrame([],columns = ['index','shcode','interval_init', 'target_day','interval_end'])
    
for i,v in df_for['index'].items():
    shcode = df_for['shcode'].iloc[i].zfill(6)
    target_date = df_for['target_day'].iloc[i]
    target_day_list = pd.to_datetime(target_date, format = "%Y-%m-%d")
    #interval calculate before 4 weeks , after 2 weeks
    interval_init_timestamp = target_day_list - datetime.timedelta(weeks=25)
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=8)
    

    # make df using series(datetime to str)
    result = pd.DataFrame(dict(interval_init = [interval_init_timestamp.strftime("%Y%m%d")],
                               target_day = [target_day_list.strftime("%Y%m%d")],
                               interval_end = [interval_end_timestamp.strftime("%Y%m%d")]))
    
    result['index'] = v
    result['shcode'] = shcode
    
    result_for_draw = result_for_draw.append(result)

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [22]:
per_df = pd.DataFrame([],columns = ['date','index','shcode'])

for i in range(len(result_for_draw)):
    df = query.query_ohlcv(shcode = result_for_draw['shcode'].iloc[i],
                         fr=result_for_draw['interval_init'].iloc[i],
                         to=result_for_draw['interval_end'].iloc[i])
    
    #데이터전처리
    df = preprocesser.sma_long(df)
    df = preprocesser.obv(df)
    df = preprocesser.rsi(df)
    df = preprocesser.stochastic(df)
    df = preprocesser.marking_cross_sma(df)
    df = preprocesser.marking_cross_rsi(df)
    df = preprocesser.marking_cross_stochastic(df)
    
    #df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d")
    df['index'] = result_for_draw['index'].iloc[i]
    df['shcode'] = result_for_draw['shcode'].iloc[i]
    df['target_day'] = result_for_draw['target_day'].iloc[i]
    
    per_df = per_df.append(df[['date','index','shcode',
                               'close_ma5', 'close_ma10', 'close_ma20', 'close_ma60','close_ma120',
                               'obv',
                               'rsi14', 'rsi14_signal', 'slowk', 'slowd', 'ma_g_c', 'ma_d_c', 'rsi_g_c', 'rsi_d_c',
                               'stoc_g_c', 'stoc_d_c']])

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the curren

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the curren

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the curren

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the curren

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the curren

...

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the curren

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the curren

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the curren

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the curren

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the curren

In [23]:
final = pd.merge(rrr, per_df, how = 'left', on = ['date','index','shcode'])

In [24]:
final[['index' ,'shcode','shname', 'target_m', 'date',  'close_ratio', 'open_', 'low', 'high', 'close','volume' ,'volume_ratio',
       'range_position', 'spot_range',  'min_target', 'max_target', 'inrange', 'spot_range_inc',
       'squeezed', 'volume_default', 'deal_f','burn_ratio','rotate_ratio', 'for_net_buy','com_net_buy','per_net_buy',
       'close_ma5', 'close_ma10', 'close_ma20', 'close_ma60','close_ma120',
       'obv',
       'rsi14', 'rsi14_signal', 'slowk', 'slowd', 'ma_g_c', 'ma_d_c', 'rsi_g_c', 'rsi_d_c',
       'stoc_g_c', 'stoc_d_c'
     ]].to_csv("af_target_day_using_weekly_v3.csv",encoding = 'ms949')